In [1]:
# Arxiv Reseach
## tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper



In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki) 
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name


'arxiv'

In [4]:
## combine tools in a agent
tools = [wiki, arxiv]

In [5]:
## Custom tools [Rag tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


d:\Langchain Projects\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Artificial_intelligence")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000206D268A020>, search_kwargs={})

In [7]:
# The new updated path
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search any query in the wikipedia page about Artificial Intelligence")
retriever_tool.name

'langsmith_search'

In [8]:
## combine tools in a agent
tools = [wiki, arxiv, retriever_tool]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Langchain Projects\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith_search', description='Search any query in the wikipedia page about Artificial Intelligence', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x000002068AEDD2D0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x00000206CFE80AF0>)]

In [ ]:
## Run all this tools woth agents and LLM Models

## Tools, LLM --> AgentExcuter


from dotenv import load_dotenv
load_dotenv()
import os
import openai
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant")

In [38]:
## Create Prompt Template
# from langsmith import Client
# client = Client()
# # from langchainhub import hub
# prompt = client.pull_prompt("ohkgi/superb_system_instruction_prompt")
# prompt.messages[0]


from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools when necessary."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])



In [39]:
## Agents
from langchain_classic.agents import AgentExecutor
from langchain_classic.agents import create_tool_calling_agent

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageCh

In [40]:
import langchain
print(langchain.__version__)

1.2.9


In [41]:
## Agent Excuter
from langchain_classic.agents import AgentExecutor
agent_excutor = AgentExecutor(agent=agent,
                            tools=tools, 
                            verbose=True)
agent_excutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[l

In [42]:
agent_excutor.invoke({"input": "Explain LangChain agents"})




> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangChain agents'}`


Agents
Main article: Agentic AI

List

Category
Philosophers category
Project
Task Force

The prevalence of generative AI tools has increased significantly since the AI boom in the 2020s. This boom was made possible by improvements in deep neural networks, particularly large language models (LLMs), which are based on the transformer architecture. Generative AI applications include chatbots such as ChatGPT, Claude, Copilot, DeepSeek, Google Gemini and Grok; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo, LTX and Sora.[172][173][174]
Companies in a variety of sectors have used generative AI, including those in software development, healthcare,[175] finance,[176] entertainment,[177] customer service,[178] sales and marketing,[179] art, writing,[180] and product design.[181]
Invoking: `langsmith_search` with `{'query': 'La

{'input': 'Explain LangChain agents',
 'output': "It seems that the information on LangChain agents is limited. However, based on the provided text, LangChain appears to be a platform for building agentic AI models. It enables developers to create AI systems that can interact with users in a more human-like way, using techniques such as dialogue management and reasoning. LangChain provides a set of tools and models that can be used to build a wide range of applications, from chatbots to virtual assistants.\n\nHere's a high-level overview of how LangChain works:\n\n1. **Dialogue Management**: LangChain uses a dialogue management system to manage the conversation flow between the user and the AI model. This system is responsible for handling user input, generating responses, and tracking the conversation context.\n2. **Reasoning**: LangChain uses a reasoning engine to enable the AI model to make decisions based on the conversation context. This engine can use a variety of techniques, inc

In [44]:
agent_excutor.invoke({"input": "What is Machine Learning?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Machine Learning'}`


Learning
Machine learning is the study of programs that can improve their performance on a given task automatically.[42] It has been a part of AI from the beginning.[e]

Deep learning
Deep learning is a subset of machine learning, which is itself a subset of artificial intelligence.[114]
Deep learning uses several layers of neurons between the network's inputs and outputs.[111] The multiple layers can progressively extract higher-level features from the raw input. For example, in image processing, lower layers may identify edges, while higher layers may identify the concepts relevant to a human such as digits, letters, or faces.[115]

Applications
Main article: Applications of artificial intelligenceAI and machine learning technology is used in most of the essential applications of the 2020s, including:
search engines (such as Google Search)
targeting online advertisements
recom

{'input': 'What is Machine Learning?',
 'output': "I couldn't find any information on Machine Learning"}

In [46]:
agent_excutor.invoke({"input": "What is the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': '1706.03762'}`
responded: It appears that the paper '1706.03762' is actually "Attention Is All You Need" not the paper you were looking for. Let me try again.



Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz

{'input': 'What is the paper 1706.03762 about?',
 'output': 'This function call was repeated 3 times.'}